# Notebook for running EfficientNet experiments
Contributions: Christoph Nötzli, Erik Norén and Sushruth Badri <br>
Comments: Erik Norén 14/12-22

# Testing EfficientNet

In [ ]:
# imports
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
from tuning import callbacks as cb
import datetime
import os
import numpy as np

In [ ]:
# image size selected according to EfficientNetB3 input size
image_size = (300,300)
batch_size = 128

# Load datasets from current _split folders
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces_split", image_size, batch_size, None, False, False)
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace_split", image_size, batch_size, None, False, False)


## Find learning rate

In [ ]:
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")

In [ ]:
epochs = 200
train_input = []
train_output = []
nr_batches = 10

model.fit(ds_train, steps_per_epoch=nr_batches, callbacks=[cb.LerningRateCallback(100, nr_batches)], epochs=epochs, verbose=2)

# Museum baseline

## Test EfficientNet Museum
Train and test EfficientNet model using Museum data set, with no pre- or in-processing bias mitigation methods applied

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight=None, weight=False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Museum.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_effnet_baseline"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

## Test EfficientNet Museum (weighted) 
Train and test EfficientNet model using Museum data set, with bias mitigation method reweighting applied

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


# Train model with class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Musuem_re.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()


with strategy.scope():
    # Build model without augmentation
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_re_effnet"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

## Test EfficientNet Museum augmented (unweighted) 
Train and test EfficientNet model using Museum data set, with bias mitigation method data set augmentation applied

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_Museum_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_Mueseum_aug.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Crossvalidate model using entire Museum data set
history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Museum_aug_effnet"
os.mkdir(dir_name)

# Evaluate the results of the crossvalidation
ev.evaluateCrossValidation(dir_name, history)

# FairFace
For transfer learning EfficientNet model is trained on the FairFace data set in order to create a base model.

## Test EfficientNet FairFace (unweighted) 
Train and test EfficientNet model using Museum data set, with no pre- or in-processing bias mitigation methods applied

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation layers
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

## Test EfficientNet FairFace (weighted) 
Train and test EfficientNet model using FairFace data set, with bias mitigation method reweighting applied

In [ ]:
epochs = 20

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model without augmentation layers
    model = m.build_model(False, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model with class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace_re.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

## Test EfficientNet FairFace augmented (unweighted) 
Train and test EfficientNet model using FairFace data set, with bias mitigation method data set augmentation applied

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Build model with augmentation layers
    model = m.build_model(True, (300,300), "Efficient")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

# Train model without class weights
utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_FairFace_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/EfficientNet_FairFace_aug.h5")

In [ ]:
# Test the model
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

# Transfer Learning
Here the experiments for performing transfer learning are performed. The base EfficientNet models trained on the FairFace data set are loaded. These models then have some of their layers frozen and is then trained again using the Museum data set.

## Transfer learning without augmentation and reweighting
Transfer learning with EfficientNet, with no pre- or in-processing bias mitigation methods applied. Trained and tested with Museum data set.

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose the directory where model EfficientNet_FairFace.h5 exists
dir_name = "XXX_XXX_EfficientNet_Fairface"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 338
utils.freezeCertainLayers(model, 334)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning with augmentation and without reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 338 
utils.freezeCertainLayers(model, 334)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_aug.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), None, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning without augmentation and with reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 338    
utils.freezeCertainLayers(model, 334)   
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_re"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_re.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_re"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning with augmentation and reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficinetNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 338    
utils.freezeCertainLayers(model, 334)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_EfficientNet_transfer_re_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/EfficientNet_transfer_re_aug.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/EfficientNet_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (300,300), class_weight, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_EfficientNet_re_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)